<div style='background-image: url("title01.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 200px'>
<div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Computers, Waves, Simulations</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Spectral Element Method - Elastic Wave Equation 1D - Homogeneous Case</div>
        </div>
    </div>
</div>

This notebook covers the following aspects:
* The weak form of the 1D elastic wave equation
* Initialization and implementataion of the Mass and Stiffness Matrices
* Solution using the finite difference scheme and its comparion with FEM solution

## Basic Equations

This notebook presents the numerical solution for the 1D elastic wave equation

\begin{equation}
\rho(x) \partial_t^2 u(x,t) = \partial_x (\mu(x) \partial_x u(x,t)) + f(x,t),
\end{equation}

using the spectral element method. This is done after a series of steps summarized as follow:

1) The wave equation is written into its weak form

2) Assume stress-free boundary condition after integration by parts 

3) Approximate the wave field as a linear combination of some basis

\begin{equation}
u(x,t) \ \approx \ \overline{u}(x,t) \ = \ \sum_{i=1}^{n} u_i(t) \ \varphi_i(x)
\end{equation}

4) Use the same basis functions in $u(x, t)$ as test functions in the weak form, the so call Galerkin principle.

6) The continuous weak form is written as a system of linear equations by considering the approximated displacement field.

\begin{equation}
\mathbf{M}^T\partial_t^2 \mathbf{u} + \mathbf{K}^T\mathbf{u} = \mathbf{f}
\end{equation}

7) Time extrapolation with centered finite differences scheme

\begin{equation}
\mathbf{u}(t + dt) = dt^2 (\mathbf{M}^T)^{-1}[\mathbf{f} - \mathbf{K}^T\mathbf{u}] + 2\mathbf{u} - \mathbf{u}(t-dt).
\end{equation}

where $\mathbf{M}$ is known as the mass matrix, and $\mathbf{K}$ the stiffness matrix.

The above solution is exactly the same presented for the classic finite-element method. Now we introduce appropriated basis functions and integration scheme to efficiently solve the system of matrices.

#### Interpolation with Lagrange Polynomials
At the elemental level, we introduce as interpolating functions the Lagrange polynomials and use $\xi$ as the space variable representing our elemental domain:

\begin{equation}
\varphi_i \ \rightarrow \ \ell_i^{(N)} (\xi) \ := \ \prod_{j \neq i}^{N+1} \frac{\xi - \xi_j}{\xi_i-\xi_j}, \qquad   i,j = 1, 2, \dotsc , N + 1  
\end{equation}

#### Numerical Integration
The integral of a continuous function $f(x)$ can be calculated after replacing $f(x)$ by a polynomial approximation that can be integrated analytically. As interpolating functions we use again the Lagrange polynomials and
obtain Gauss-Lobatto-Legendre quadrature. Here, the GLL points are used to perform the integral. 

\begin{equation}
\int_{-1}^1 f(x) \ dx \approx \int _{-1}^1 P_N(x) dx = \sum_{i=1}^{N+1}
 w_i f(x_i) 
\end{equation}





In [ ]:
# This is a configuration step for the exercise. Please run it before plotting!
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import math
from IPython.display import display, HTML
from progress_bar import ProgressBarHandler
from ricker import ricker
from gll import gll
from lagrange1st import lagrange1st

### 1. Initialization of setup

In [2]:
# Initialization of setup
# ---------------------------------------------------------------
nt    = 600         # number of time steps
xmax  = 10000.        # Length of domain [m]
vs    = 2500.         # S velocity [m/s]
rho   = 2000          # Density [kg/m^3]
mu    = rho * vs**2   # Shear modulus mu
N     = 5           # Order of Lagrange polynomials
ne    = 125           # Number of elements
Tdom  = .1            # Dominant period of Ricker source wavelet
idisp = 5            # Plotting each iplot snapshot

# variables for elemental matrices
Me = np.zeros(N+1, dtype =  float)
Ke = np.zeros((N+1, N+1), dtype =  float)
# ----------------------------------------------------------------

# Initialization of GLL points integration weights
[xi, w] = gll(N)    # xi, N+1 coordinates [-1 1] of GLL points
                    # w Integration weights at GLL locations
# Space domain
le = xmax/ne        # Length of elements
# Vector with GLL points  
k = 0

# Initialization of physical coordinates xg in the entire space (commment added May 15, 2020)
xg = np.zeros((N*ne)+1) 
xg[k] = 0
for i in range(1,ne+1):
    for j in range(0,N):
        k = k+1
        xg[k] = (i-1)*le + .5*(xi[j+1]+1)*le

# ---------------------------------------------------------------
dxmin = min(np.diff(xg))  
eps = 0.2           # Courant value
dt = eps*dxmin/vs   # Global time step

# Mapping - Jacobian
J = le/2 
Ji = 1/J    # Inverse Jacobian

# 1st derivative of Lagrange polynomials
l1d = lagrange1st(N)   # Array with GLL as columns for each N+1 polynomial


In [ ]:
# --- Assembly & sizes (must run before mass/stiffness plots and time loop) ---
ng = N*ne + 1

M  = np.zeros((ng, ng))
K  = np.zeros((ng, ng))
Minv = None

J  = le/2.0
Ji = 1.0/J
W  = np.diag(w)

# Homogeneous rho, mu (scalars)
Me = (rho * J) * W                       # (N+1)x(N+1)
Ke = (mu * (Ji**2) * J) * (l1d.T @ W @ l1d)

for e in range(ne):
    idx = e*N + np.arange(N+1)
    M[np.ix_(idx, idx)] += Me
    K[np.ix_(idx, idx)] += Ke

# Mass-lumped inverse (diagonal)
Minv = np.diag(1.0 / np.diag(M))

### 2. The Mass Matrix
Now we initialize the mass and stiffness matrices. In general, the mass matrix at the elemental level is given

\begin{equation}
M_{ji}^e \ = \ w_j  \ \rho (\xi)  \ \frac{\mathrm{d}x}{\mathrm{d}\xi} \delta_{ij} \vert_ {\xi = \xi_j}
\end{equation}


We implement the mass matrix using the integration weights at GLL locations $w$, the jacobian $J$, and density $\rho$. Then, perform the global assembly of the mass matrix, compute its inverse, and display the inverse mass matrix to visually inspect how it looks like.

In [ ]:
# Plot mass matrix visualization
plt.figure(figsize=(6,5))
plt.imshow(M, cmap='viridis')
plt.colorbar()
plt.title(r'Mass Matrix $\mathbf{M}$')
plt.show()

### 3. The Stiffness matrix
On the other hand, the general form of the stiffness matrix at the elemtal level is

\begin{equation}
K_{ji}^e \ = \ \sum_{k = 1}^{N+1} w_k \mu (\xi) \partial_\xi \ell_j (\xi) \partial_\xi \ell_i (\xi) \left(\frac{\mathrm{d}\xi}{\mathrm{d}x} \right)^2 \frac{\mathrm{d}x}{\mathrm{d}\xi} \vert_{\xi = \xi_k}
\end{equation} 

We implement the stiffness matrix using the integration weights at GLL locations $w$, the jacobian $J$, and shear stress $\mu$. Then, perform the global assembly of the mass matrix and display the matrix to visually inspect how it looks like.

In [ ]:
# Plot stiffness matrix visualization
plt.figure(figsize=(6,5))
plt.imshow(K, cmap='viridis')
plt.colorbar()
plt.title(r'Stiffness Matrix $\mathbf{K}$')
plt.show()

### 4. Spectral Element solution 

Finally we implement the spectral element solution using the computed mass $M$ and stiffness $K$ matrices together with a finite differences extrapolation scheme

\begin{equation}
\mathbf{u}(t + dt) = dt^2 (\mathbf{M}^T)^{-1}[\mathbf{f} - \mathbf{K}^T\mathbf{u}] + 2\mathbf{u} - \mathbf{u}(t-dt).
\end{equation}

In [ ]:
# SE Solution, Time extrapolation
# ---------------------------------------------------------------

# initialize source time function and force vector f
src  = ricker(dt,Tdom)
isrc = int(np.floor(ng/2))   # Source location

# Initialization of solution vectors
u = np.zeros(ng)
uold = u
unew = u
f = u 

# --------------------------------------------------------------
# Initialize animated plot
# --------------------------------------------------------------
fig1, ax1 = plt.subplots(figsize=(10,6))
line1, = ax1.plot(xg, u, lw=1.5)
ax1.set_title('SEM 1D Animation', size=24)
ax1.set_xlabel(' x (m)', size=24)
ax1.set_ylabel(' Amplitude ', size=24)
ax1.tick_params(axis='x', labelsize=16)
ax1.tick_params(axis='y', labelsize=16)

plt.show()

In [ ]:
# ---------------------------------------------------------------
# Time extrapolation
# ---------------------------------------------------------------

# Progress Bar
progress_handler = ProgressBarHandler(nt, "Finite elements 1D Animation")

# Simulation Results
u_results = np.zeros((math.ceil(nt/idisp), (N*ne)+1))

for it in range(nt): 
    # Source initialization
    f= np.zeros(ng)
    if it < len(src):
        f[isrc-1] = src[it-1] 
               
    # Time extrapolation
    unew = dt**2 * Minv @ (f - K @ u) + 2 * u - uold
    uold, u = u, unew

    # --------------------------------------   
    # Save Results
    if it % idisp == 0:
        u_results[int(it/idisp), :] = u

    progress_handler(it)

None

In [ ]:
from animation import create_animation_se_homo_1d_solution

ani = create_animation_se_homo_1d_solution(locals())

display(HTML(ani.to_jshtml()))